In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

from sklearn.metrics import accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


In [192]:
# get preprocessed data
df_train_encoded = pd.read_csv('cleaned_train_data_encoded.csv')
df_submission_encoded = pd.read_csv('cleaned_submission_data_encoded.csv')
df_submission_encoded.drop('num', axis=1, inplace=True)

df_train = pd.read_csv('cleaned_train_data.csv')
df_submission = pd.read_csv('cleaned_submission_data.csv')
df_submission.drop('num', axis=1, inplace=True)

combine = [df_train, df_train_encoded, df_submission, df_submission_encoded]

In [193]:
df_train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,num,chol*trestbps
0,2.0,1.0,2.0,2.0,3.0,0.0,0.0,160.0,0.0,1.0,1.0,6
1,2.0,1.0,2.0,1.0,2.0,0.0,0.0,100.0,0.0,0.0,0.0,2
2,2.0,0.0,2.0,2.0,1.0,0.0,0.0,180.0,0.0,0.0,0.0,2
3,3.0,1.0,4.0,2.0,1.0,0.0,2.0,106.0,1.0,1.0,1.0,2
4,3.0,1.0,4.0,3.0,3.0,0.0,0.0,155.0,0.0,0.0,0.0,9


In [194]:
# scale data
scale_col = ['thalach']

scaler = StandardScaler()
df_train[scale_col] = scaler.fit_transform(df_train[scale_col])

scaler = StandardScaler()
df_submission[scale_col] = scaler.fit_transform(df_submission[scale_col])

scaler = StandardScaler()
df_train_encoded[scale_col] = scaler.fit_transform(df_train_encoded[scale_col])

scaler = StandardScaler()
df_submission_encoded[scale_col] = scaler.fit_transform(df_submission_encoded[scale_col])

### for seeing what is best classifier, we will use train test split. once we choose a classifier, we retrain on entire train data

In [220]:
# separate prediction column
X_encoded = df_train_encoded.drop('num', axis=1)
X = df_train.drop('num', axis=1)
y = df_train['num']

X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded, y, test_size=0.3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Logistic Regression 

In [221]:
# using default solver liblinear because dataset is small

log = LogisticRegression().fit(X_train, y_train)

In [222]:
# predict testing data
pred = log.predict(X_test)
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

        0.0       0.84      0.74      0.79       117
        1.0       0.42      0.58      0.49        38

avg / total       0.74      0.70      0.72       155



In [223]:
# encoded

log = LogisticRegression().fit(X_train_encoded, y_train_encoded)
pred = log.predict(X_test_encoded)
print(classification_report(pred, y_test_encoded))

             precision    recall  f1-score   support

        0.0       0.90      0.84      0.87       119
        1.0       0.57      0.69      0.62        36

avg / total       0.82      0.81      0.81       155



## Random Forest

In [224]:
forest = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)

In [225]:
# predict testing data
pred = forest.predict(X_test)
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

        0.0       0.85      0.75      0.80       118
        1.0       0.42      0.59      0.49        37

avg / total       0.75      0.71      0.72       155



In [226]:
# encoded

forest = RandomForestClassifier(n_estimators=100).fit(X_train_encoded, y_train_encoded)
pred = forest.predict(X_test_encoded)
print(classification_report(pred, y_test_encoded))

             precision    recall  f1-score   support

        0.0       0.84      0.81      0.82       115
        1.0       0.50      0.55      0.52        40

avg / total       0.75      0.74      0.75       155



## SVM

In [227]:
# fit model with optimal hyper parameters
param_grid = {'C': [0.1, 1, 10, 100, 1000],'gamma': [1,0.1,0.01,0.001,0.0001]} 
svm = GridSearchCV(SVC(), param_grid=param_grid, verbose=3).fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.694215, total=   0.0s
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.694215, total=   0.0s
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.697479, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.694215, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.694215, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.697479, total=   0.0s
[CV] gamma=0.01, C=0.1 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ............... gamma=0.0001, C=10, score=0.697479, total=   0.0s
[CV] gamma=1, C=100 ..................................................
[CV] ................... gamma=1, C=100, score=0.752066, total=   0.0s
[CV] gamma=1, C=100 ..................................................
[CV] ................... gamma=1, C=100, score=0.752066, total=   0.0s
[CV] gamma=1, C=100 ..................................................
[CV] ................... gamma=1, C=100, score=0.764706, total=   0.0s
[CV] gamma=0.1, C=100 ................................................
[CV] ................. gamma=0.1, C=100, score=0.669421, total=   0.0s
[CV] gamma=0.1, C=100 ................................................
[CV] ................. gamma=0.1, C=100, score=0.776860, total=   0.0s
[CV] gamma=0.1, C=100 ................................................
[CV] ................. gamma=0.1, C=100, score=0.747899, total=   0.0s
[CV] gamma=0.01, C=100 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.3s finished


In [228]:
svm.best_params_

{'C': 1, 'gamma': 0.1}

In [229]:
pred = svm.predict(X_test)
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

        0.0       0.86      0.74      0.80       120
        1.0       0.40      0.60      0.48        35

avg / total       0.76      0.71      0.73       155



In [230]:
# encoded

# fit model with optimal hyper parameters
param_grid = {'C': [0.1, 1, 10, 100, 1000],'gamma': [1,0.1,0.01,0.001,0.0001]} 
svm = GridSearchCV(SVC(), param_grid=param_grid, verbose=3).fit(X_train_encoded, y_train_encoded)
pred = svm.predict(X_test_encoded)
print(classification_report(pred, y_test_encoded))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.669421, total=   0.0s
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.675000, total=   0.0s
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.675000, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.669421, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.675000, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.675000, total=   0.0s
[CV] gamma=0.01, C=0.1 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................. gamma=0.1, C=10, score=0.741667, total=   0.0s
[CV] gamma=0.1, C=10 .................................................
[CV] .................. gamma=0.1, C=10, score=0.775000, total=   0.0s
[CV] gamma=0.01, C=10 ................................................
[CV] ................. gamma=0.01, C=10, score=0.735537, total=   0.0s
[CV] gamma=0.01, C=10 ................................................
[CV] ................. gamma=0.01, C=10, score=0.800000, total=   0.0s
[CV] gamma=0.01, C=10 ................................................
[CV] ................. gamma=0.01, C=10, score=0.783333, total=   0.0s
[CV] gamma=0.001, C=10 ...............................................
[CV] ................ gamma=0.001, C=10, score=0.719008, total=   0.0s
[CV] gamma=0.001, C=10 ...............................................
[CV] ................ gamma=0.001, C=10, score=0.733333, total=   0.0s
[CV] gamma=0.001, C=10 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.4s finished


## keras

In [231]:
# encoded

model_encoded = Sequential()
model_encoded.add(Dense(units=10, input_dim=22, activation='relu'))
model_encoded.add(Dropout(0.2))
model_encoded.add(Dense(units=10, activation='relu'))
model_encoded.add(Dropout(0.2))
model_encoded.add(Dense(units=10, activation='relu'))
model_encoded.add(Dropout(0.2))
model_encoded.add(Dense(units=1, activation='sigmoid'))

model_encoded.compile(optimizer='rmsprop',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

In [232]:
model_encoded.fit(X_train_encoded.as_matrix(), y_train_encoded.as_matrix(), epochs=200, validation_split=0.2)

Train on 288 samples, validate on 73 samples
Epoch 1/200
288/288 [==============================] - 0s - loss: 0.7047 - acc: 0.5069 - val_loss: 0.7018 - val_acc: 0.4521
Epoch 2/200
288/288 [==============================] - 0s - loss: 0.6977 - acc: 0.5243 - val_loss: 0.6875 - val_acc: 0.6438
Epoch 3/200
288/288 [==============================] - 0s - loss: 0.6858 - acc: 0.5833 - val_loss: 0.6749 - val_acc: 0.6438
Epoch 4/200
288/288 [==============================] - 0s - loss: 0.6726 - acc: 0.6424 - val_loss: 0.6654 - val_acc: 0.7260
Epoch 5/200
288/288 [==============================] - 0s - loss: 0.6731 - acc: 0.6285 - val_loss: 0.6550 - val_acc: 0.7260
Epoch 6/200
288/288 [==============================] - 0s - loss: 0.6646 - acc: 0.6285 - val_loss: 0.6472 - val_acc: 0.7123
Epoch 7/200
288/288 [==============================] - 0s - loss: 0.6669 - acc: 0.6354 - val_loss: 0.6377 - val_acc: 0.7945
Epoch 8/200
288/288 [==============================] - 0s - loss: 0.6481 - acc: 0.7118 

In [233]:
model_encoded.evaluate(X_test_encoded.as_matrix(), y_test_encoded.as_matrix())

 32/155 [=====>........................] - ETA: 0s

[0.47851890017909388, 0.78064515706031556]

In [234]:
model = Sequential()
model.add(Dense(units=10, input_dim=11, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

model.fit(X_train.as_matrix(), y_train.as_matrix(), epochs=200, validation_split=0.2)

Train on 288 samples, validate on 73 samples
Epoch 1/200
288/288 [==============================] - 0s - loss: 0.8026 - acc: 0.4410 - val_loss: 0.6942 - val_acc: 0.5890
Epoch 2/200
288/288 [==============================] - 0s - loss: 0.7443 - acc: 0.5312 - val_loss: 0.6682 - val_acc: 0.6438
Epoch 3/200
288/288 [==============================] - 0s - loss: 0.7345 - acc: 0.5417 - val_loss: 0.6521 - val_acc: 0.7123
Epoch 4/200
288/288 [==============================] - 0s - loss: 0.7000 - acc: 0.5972 - val_loss: 0.6425 - val_acc: 0.7123
Epoch 5/200
288/288 [==============================] - 0s - loss: 0.6886 - acc: 0.6111 - val_loss: 0.6353 - val_acc: 0.7123
Epoch 6/200
288/288 [==============================] - 0s - loss: 0.6558 - acc: 0.6632 - val_loss: 0.6276 - val_acc: 0.7123
Epoch 7/200
288/288 [==============================] - 0s - loss: 0.6352 - acc: 0.6944 - val_loss: 0.6223 - val_acc: 0.7123
Epoch 8/200
288/288 [==============================] - 0s - loss: 0.6654 - acc: 0.6354 

In [235]:
model.evaluate(X_test.as_matrix(), y_test.as_matrix())

 32/155 [=====>........................] - ETA: 0s

[0.55574163775290208, 0.71612902918169574]

# retrain model on entire data set for kaggle submission

# going to create 8 prediction files for log. reg., random forest, svm, and keras just to see how each really do

## random forest

In [236]:
forest = RandomForestClassifier(n_estimators=100).fit(X, y)
forest_pred = forest.predict(df_submission.drop('id', axis=1))

kaggle_forest = pd.DataFrame({
        'id': df_submission['id'],
        'num': forest_pred
    }, dtype=np.int32)

kaggle_forest.to_csv('kaggle_forest_pred.csv', index=False)

In [238]:
# encoded

forest = RandomForestClassifier(n_estimators=100).fit(X_encoded, y)
forest_pred = forest.predict(df_submission_encoded.drop('id', axis=1))

In [239]:
kaggle_forest = pd.DataFrame({
        'id': df_submission_encoded['id'],
        'num': forest_pred
    }, dtype=np.int32)

In [240]:
kaggle_forest.to_csv('kaggle_forest_pred_encoded.csv', index=False)

In [241]:
kaggle_forest['num'].value_counts()

0    62
1    38
Name: num, dtype: int64

## svm

In [209]:
param_grid = {'C': [0.1, 0.5, 1, 5, 10, 50, 100],'gamma': [1,0.1,0.01,0.001,0.0001]} 

In [242]:
svm = GridSearchCV(SVC(), param_grid=param_grid).fit(X, y)
svm_pred = svm.predict(df_submission.drop('id', axis=1))

kaggle_svm = pd.DataFrame({
        'id': df_submission['id'],
        'num': svm_pred,
    }, dtype=np.int32)

kaggle_svm.to_csv('kaggle_svm_pred.csv', index=False)

In [243]:
# encoded

svm = GridSearchCV(SVC(), param_grid=param_grid).fit(X_encoded, y)

In [244]:
svm_pred = svm.predict(df_submission_encoded.drop('id', axis=1))

In [245]:
kaggle_svm = pd.DataFrame({
        'id': df_submission_encoded['id'],
        'num': svm_pred,
    }, dtype=np.int32)

In [246]:
kaggle_svm.to_csv('kaggle_svm_pred_encoded.csv', index=False)

In [247]:
kaggle_svm['num'].value_counts()

0    54
1    46
Name: num, dtype: int64

## keras

In [248]:
model.fit(X.as_matrix(), y.as_matrix(), epochs=400)

Epoch 1/400
516/516 [==============================] - 0s - loss: 0.5016 - acc: 0.7733     
Epoch 2/400
516/516 [==============================] - 0s - loss: 0.5121 - acc: 0.7636     
Epoch 3/400
516/516 [==============================] - 0s - loss: 0.5103 - acc: 0.7674     
Epoch 4/400
516/516 [==============================] - 0s - loss: 0.4964 - acc: 0.7771     
Epoch 5/400
516/516 [==============================] - 0s - loss: 0.4947 - acc: 0.7752     
Epoch 6/400
516/516 [==============================] - 0s - loss: 0.5024 - acc: 0.7616     
Epoch 7/400
516/516 [==============================] - 0s - loss: 0.4802 - acc: 0.7829     
Epoch 8/400
516/516 [==============================] - 0s - loss: 0.4983 - acc: 0.7810     
Epoch 9/400
516/516 [==============================] - 0s - loss: 0.4832 - acc: 0.7810     
Epoch 10/400
516/516 [==============================] - 0s - loss: 0.4969 - acc: 0.7810     
Epoch 11/400
516/516 [==============================] - 0s - loss: 0.5134 - acc

In [249]:
keras_pred = model.predict_classes(df_submission.drop('id', axis=1).as_matrix())

 32/100 [========>.....................] - ETA: 0s

In [250]:
keras_pred = keras_pred.flatten()
kaggle_keras = pd.DataFrame({
        'id': df_submission['id'],
        'num': keras_pred
    },dtype=np.int32)
kaggle_keras.to_csv('kaggle_keras_pred.csv', index=False)

In [251]:
kaggle_keras['num'].value_counts()

0    61
1    39
Name: num, dtype: int64

In [ ]:
# encoded

In [252]:
model_encoded.fit(X_encoded.as_matrix(), y.as_matrix(), epochs=400)

Epoch 1/400
516/516 [==============================] - 0s - loss: 0.4493 - acc: 0.8159     
Epoch 2/400
516/516 [==============================] - 0s - loss: 0.4393 - acc: 0.8023     
Epoch 3/400
516/516 [==============================] - 0s - loss: 0.4749 - acc: 0.8062     
Epoch 4/400
516/516 [==============================] - 0s - loss: 0.4100 - acc: 0.8295     
Epoch 5/400
516/516 [==============================] - 0s - loss: 0.4244 - acc: 0.8159     
Epoch 6/400
516/516 [==============================] - 0s - loss: 0.4377 - acc: 0.8198     
Epoch 7/400
516/516 [==============================] - 0s - loss: 0.4186 - acc: 0.8314     
Epoch 8/400
516/516 [==============================] - 0s - loss: 0.4223 - acc: 0.8198     
Epoch 9/400
516/516 [==============================] - 0s - loss: 0.4119 - acc: 0.8256     
Epoch 10/400
516/516 [==============================] - 0s - loss: 0.4293 - acc: 0.8120     
Epoch 11/400
516/516 [==============================] - 0s - loss: 0.4334 - acc

In [253]:
keras_pred = model_encoded.predict_classes(df_submission_encoded.drop('id', axis=1).as_matrix())

 32/100 [========>.....................] - ETA: 0s

In [254]:
keras_pred = keras_pred.flatten()
kaggle_keras = pd.DataFrame({
        'id': df_submission_encoded['id'],
        'num': keras_pred
    },dtype=np.int32)
kaggle_keras.to_csv('kaggle_keras_pred_encoded.csv', index=False)

## log. reg. 

In [255]:
log = LogisticRegression().fit(X, y)
log_pred = log.predict(df_submission.drop('id', axis=1))

kaggle_log = pd.DataFrame({
        'id': df_submission['id'],
        'num': log_pred
    }, dtype=np.int32)

kaggle_log.to_csv('kaggle_log_pred.csv', index=False)

# encoded
log = LogisticRegression().fit(X_encoded, y)
log_pred = log.predict(df_submission_encoded.drop('id', axis=1))

kaggle_log = pd.DataFrame({
        'id': df_submission_encoded['id'],
        'num': log_pred
    }, dtype=np.int32)

kaggle_log.to_csv('kaggle_log_pred_encoded.csv', index=False)

# save model

In [41]:
from sklearn.externals import joblib
joblib.dump(log, 'heartdisease_model.pkl') 

['heartdisease_model.pkl']